In [380]:
class Person(object):
    def __init__(self):
        self.name = 0
        self.age = 0
        self.gender = 0

In [381]:
import copy
class Josephus:
    '约瑟夫环'

    def __init__(self):
        self.people = []
        self.step = 0
        self.start = 1

    def append(self, obj):
        self.people.append(obj)

    def pop(self):
        self.people.pop(0)

    def query_list(self):
        return self.people

    def len_list(self):
        return len(self.people)

    def reset(self):
        self.people = []
        self.step = 0
        self.start = 1

    def next_bymod(self):
        temp = copy.copy(self.people)
        length = len(temp)
        if length == 0:
            return None
        id_ = self.start

        for i in range(length):
            id_ = (id_ + self.step - 1) % len(temp)
            index = temp.pop(id_)
            yield index

    def next_circular(self):
        temp = copy.copy(self.people)
        if len(temp) == 0:
            return None

        index = 1
        start = self.start % len(temp)
        temp = temp[start:] + temp[:start]

        while index:
            head = (self.step-1) % len(temp)
            temp = temp[head:] + temp[:head]
            index = temp.pop(0)
            yield index

    def read_txt(self, file_address):
        file_ = open(file_address, 'r', encoding = 'utf-8')
        #文本文档采用的是gbk编码，需要转换成python能识别的编码，比如utf-8

        line = file_.readline().strip()
        while line:
            index_person = line.split(',')
            person = create_person(index_person[0], index_person[1], index_person[2])
            Josephus.append(self, person)
            line = file_.readline().strip()




In [382]:
def create_person(name, age, gender):
    obj = Person()
    obj.name = name
    obj.age = age 
    obj.gender = gender
    
    return obj
    

In [383]:
ring = Josephus()
ring.reset()
ring.start = 6
ring.step = 3

# ring.append(create_person('Morty', 12, '男'))
# ring.append(create_person('Rick', 53, '男'))
# ring.append(create_person('Summer', 16, '女'))
# ring.append(create_person('Beth', 34, '女'))
# ring.append(create_person('Jerry', 36, '男'))

#读取txt
ring.read_txt('C:\\Users\\76747\\Desktop\\Python\\practice\\people.txt')

In [384]:
length = ring.len_list()

generator_bymod = ring.next_bymod()
for i in range(length):
    index = next(generator_bymod)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, index.name, index.age, index.gender))

print('-'*30)

generator_circular = ring.next_circular()
for i in range(length):
    index = next(generator_circular)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, index.name, index.age, index.gender))

第1个出列的人:Beth；年龄: 34；性别: 女
第2个出列的人:Rick；年龄: 53；性别: 男
第3个出列的人:Morty；年龄: 12；性别: 男
第4个出列的人:Summer；年龄: 16；性别: 女
第5个出列的人:Jerry；年龄: 36；性别: 男
------------------------------
第1个出列的人:Beth；年龄: 34；性别: 女
第2个出列的人:Rick；年龄: 53；性别: 男
第3个出列的人:Morty；年龄: 12；性别: 男
第4个出列的人:Summer；年龄: 16；性别: 女
第5个出列的人:Jerry；年龄: 36；性别: 男
